# Исследование MyAdamW и AdamW

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime as dt
import pandas as pd

In [2]:
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
dataset = "mushrooms.txt" 
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]
n, d = X.shape
print(X.shape, y.shape)
y = y*2 - 3
print(y)

(8124, 112) (8124,)
[-1.  1.  1. ...  1. -1.  1.]


## Загрузка данных

### Wine

In [8]:
data = pd.read_csv('Wine.csv')
data['color'] = data['color'].apply(lambda x: 1 if x == 'red' else -1)
X = data.drop(columns='color').to_numpy()
y = data['color'].to_numpy()
X = np.array(X, dtype=np.float64)

### Water PROBABILITY

In [23]:
data = pd.read_csv('water_potability.csv')
data = data.dropna(axis=0)
X = data.drop(columns='Potability').to_numpy()
X = np.array(X, dtype=np.float64)
y = data['Potability'].to_numpy()
y = np.array(y, dtype=np.float64)
y = y * 2 - 1

### Diabete

In [43]:
data = pd.read_csv('diabetes.csv')
data = data.dropna(axis=0)
X = data.drop(columns='1').to_numpy()
X = np.array(X, dtype=np.float64)
y = data['1'].to_numpy()
y = np.array(y, dtype=np.float64)
y = y * 2 - 1

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape

(6499, 112)

In [4]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

In [5]:
from importlib import reload
import lib
reload(lib)
from lib  import MyLogregression

In [6]:
def plot_models_acc_error(*args):
    _, ax = plt.subplots(2, 2, figsize=(14, 12))
    title = args[1]
    ax[0, 0].set_title(f"Логарифм ошибки от номера итерации ($error = || {title} ||$)")
    ax[0, 0].set_xlabel("Номер итерации")
    ax[0, 0].set_ylabel("Логарифм ошибки")
    
    ax[0, 1].set_title(f"Логарифм ошибки от времени ($error = || {title}||$)")
    ax[0, 1].set_xlabel("Время, с")
    ax[0, 1].set_ylabel("Логарифм ошибки")
    
    ax[1, 0].set_title("Точность от номера итерации")
    ax[1, 0].set_xlabel("Номер итерации")
    ax[1, 0].set_ylabel("Точность")
    
    ax[1, 1].set_title("Точность от времени")
    ax[1, 1].set_xlabel("Время, с")
    ax[1, 1].set_ylabel("Точность")
    for model in args[0]:

        y = np.array(model.get_errors())
        x = np.linspace(0, len(y), len(y))
        ax[0, 0].plot(x, np.log(y), label=model.get_name())
        x = np.array(model.get_time())
        ax[0, 1].plot(x, np.log(y), label=model.get_name())
        if (model.get_name()[0:5] == "AdamW"):
            y = np.array(model._error_adamw)
            x = np.linspace(0, len(y), len(y))
            ax[0, 0].plot(x, np.log(y), label=model.get_name() + '_true')
            x = np.array(model.get_time())
            ax[0, 1].plot(x, np.log(y), label=model.get_name() + '_true')

        y = np.array(model.get_accuracy())
        x = np.linspace(0, len(y), len(y))
        ax[1, 0].plot(x, y, label=model.get_name())
        x = np.array(model.get_time())
        ax[1, 1].plot(x, y, label=model.get_name())


    legend_box = ax[1, 1].legend(framealpha=1).get_frame()
    legend_box.set_facecolor("white")
    legend_box.set_edgecolor("black")

    legend_box = ax[1, 0].legend(framealpha=1).get_frame()
    legend_box.set_facecolor("white")
    legend_box.set_edgecolor("black")

    legend_box = ax[0, 1].legend(framealpha=1).get_frame()
    legend_box.set_facecolor("white")
    legend_box.set_edgecolor("black")

    legend_box = ax[0, 0].legend(framealpha=1).get_frame()
    legend_box.set_facecolor("white")
    legend_box.set_edgecolor("black")


def plot_models_acc_error2(*args):
    _, ax = plt.subplots(1, 2, figsize=(16, 9))
    title = args[1]
    ax[0].set_title(f"Логарифм ошибки от номера итерации ($error = || {title} ||$)")
    ax[0].set_xlabel("Номер итерации")
    ax[0].set_ylabel("Логарифм ошибки")
    
    ax[1].set_title("Точность от номера итерации")
    ax[1].set_xlabel("Номер итерации")
    ax[1].set_ylabel("Точность")

    for model in args[0]:

        if (model.get_name()[0:5] == "AdamW"):
            y = np.array(model.get_errors())
            x = np.linspace(0, len(y), len(y))
            ax[0].plot(x, np.log(y), label=model.get_name() + '[ $\\nabla f + \\nabla r$ ]')
            
            y = np.array(model._error_adamw)
            x = np.linspace(0, len(y), len(y))
            ax[0].plot(x, np.log(y), label=model.get_name() + '[ $\\nabla f + \\nabla r D_t$ ]')
        else:
            y = np.array(model.get_errors())
            x = np.linspace(0, len(y), len(y))
            ax[0].plot(x, np.log(y), label=model.get_name())

        y = np.array(model.get_accuracy())
        x = np.linspace(0, len(y), len(y))
        ax[1].plot(x, y, label=model.get_name())
        
    legend_box = ax[1].legend(framealpha=1).get_frame()
    legend_box.set_facecolor("white")
    legend_box.set_edgecolor("black")

    legend_box = ax[0].legend(framealpha=1).get_frame()
    legend_box.set_facecolor("white")
    legend_box.set_edgecolor("black")

### GD

In [7]:
def get_gd_models(X_train, y_train, X_test, y_test, learning_rates=np.linspace(1, 0.0001, 5)):
    gd_models = []
    for lr in learning_rates:
        GD = MyLogregression(iter=1000, lr_func=lambda w: lr, name="GD", label="GD(%0.4f)" % lr)
        GD.fit(X_train, y_train, X_test, y_test)
        gd_models.append(GD)
    return gd_models

def get_adamw_models(X_train, y_train, X_test, y_test, \
                    learning_rates=[0.1, 0.05, 0.01], weight_decays=[0.1, 0.05, 0.005]):
    adamw_models = []
    for lr in learning_rates:
        for wd in weight_decays:
            AdamW = MyLogregression(iter=1000, lr_func=lambda w: lr, name="AdamW", label=f"AdamW({lr}, {wd})", betas=[0.99, 0.999], l2_coef=wd)
            AdamW.fit(X_train, y_train, X_test, y_test)    
            adamw_models.append(AdamW)
    return adamw_models

def get_adaml2_models(X_train, y_train, X_test, y_test, \
                      learning_rates=[0.1, 0.05, 0.01], weight_decays=[0.1, 0.05, 0.005]):
    adaml2_models = []
    for lr in learning_rates:
        for wd in weight_decays:
            Adaml2 = MyLogregression(iter=1000, lr_func=lambda w: lr, name="AdamL2", label=f"AdamL2({lr}, {wd})", betas=[0.99, 0.999], l2_coef=wd)
            Adaml2.fit(X_train, y_train, X_test, y_test)    
            adaml2_models.append(Adaml2)
    return adaml2_models

def get_oasis_models(X_train, y_train, X_test, y_test, \
                    learning_rates=[0.1, 0.05, 0.01], weight_decays=[0.1, 0.05, 0.005]):
    oasis_models = []
    for lr in learning_rates:
        for wd in weight_decays:
            OASIS = MyLogregression(iter=1000, lr_func=lambda w: lr, name="OASIS", label=f"OASIS({lr}, {wd})", betas=[0.99, 0.999], l2_coef=wd)
            OASIS.fit(X_train, y_train, X_test, y_test)    
            oasis_models.append(OASIS)
    return oasis_models

### AdamW

In [12]:
learning_rates = [0.1, 0.05, 0.01]
weight_decays = [0.1, 0.05, 0.005]
adamw_models = get_adamw_models(X_train, y_train, X_test, y_test, learning_rates, weight_decays)

KeyboardInterrupt: 

In [ ]:
plot_models_acc_error2(adamw_models[0:5], '\\nabla f(x)')
plot_models_acc_error2(adamw_models[5:], '\\nabla f(x)')

In [69]:
learning_rates = [0.1, 0.05, 0.01]
weight_decays = [0.1, 0.05, 0.005]
adaml2_models = get_adaml2_models(X_train, y_train, X_test, y_test, learning_rates, weight_decays)

In [ ]:
learning_rates = [0.1, 0.05, 0.01]
weight_decays = [0.1, 0.05, 0.005]
gd_models = get_gd_models(X_train, y_train, X_test, y_test, learning_rates)

In [ ]:
learning_rates = [0.1, 0.05, 0.01]
weight_decays = [0.1, 0.05, 0.005]
oasis_models = get_oasis_models(X_train, y_train, X_test, y_test, learning_rates)

In [ ]:
plot_models_acc_error2(adaml2_models[1:], '\\nabla f(x) + \\nabla r(x)')

In [ ]:
plot_models_acc_error(oasis_models, '\\nabla f(x)')

# Experiments with Wine:

### Data download

In [9]:
data = pd.read_csv('Wine.csv')
data['color'] = data['color'].apply(lambda x: 1 if x == 'red' else -1)
X = data.drop(columns='color').to_numpy()
y = data['color'].to_numpy()
X = np.array(X, dtype=np.float64)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape

(5197, 12)

In [11]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)